In [8]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle

from pympler import asizeof
import gpflow
import tensorflow_probability as tfp

from matplotlib import pyplot as plt

In [14]:
#Make Simulator and Training Data
cs_name_val = 10
noise_mean = 0
noise_std = None
seed = 1
retrain_GP = 5
normalize = True
#Define method, ep_enum classes, indecies to consider, and kernel
meth_name = Method_name_enum(3)
method = GPBO_Methods(meth_name)
gen_meth_theta = Gen_meth_enum(1)
gen_meth_x = Gen_meth_enum(2)
num_x_data = 5

#Define Simulator Class (Export your Simulator Object Here)
simulator = simulator_helper_test_fxns(cs_name_val, noise_mean, noise_std, seed)
num_theta_data = len(simulator.indeces_to_consider)*10
#Generate Exp Data (OR Add your own experimental data as a Data class object)
set_seed = 1 #Set set_seed to 1 for data generation
gen_meth_x = Gen_meth_enum(gen_meth_x)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x, set_seed)
#Set simulator noise_std artifically as 5% of y_exp mean (So that noise will be set rather than trained)
simulator.noise_std = np.abs(np.mean(exp_data.y_vals))*0.05
print(simulator.noise_std)
#Note at present, training data is always the same between jobs since we set the data generation seed to 1
sim_data = simulator.gen_sim_data(num_theta_data, num_x_data, gen_meth_theta, gen_meth_x, 1.0, seed, False)
val_data = simulator.gen_sim_data(10, 10, Gen_meth_enum(1), Gen_meth_enum(1), 1.0, seed + 1, False)
# print(sim_data.theta_vals)

noise_std = simulator.noise_std #Yexp_std is exactly the noise_std of the GP Kernel

if method.emulator == True:
    all_gp_data = sim_data
    all_val_data = val_data
    gp_object = Type_2_GP_Emulator(all_gp_data, all_val_data, None, None, None, Kernel_enum(1), None, noise_std, None, 
                                retrain_GP, seed, normalize, None, None, None, None)
else:
    all_gp_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data, 1.0, False)
    print(len(all_gp_data.theta_vals), len(all_gp_data.x_vals), len(all_gp_data.y_vals))
    all_val_data = simulator.sim_data_to_sse_sim_data(method, val_data, exp_data, 1.0, False)
    print(len(all_val_data.theta_vals), len(all_val_data.x_vals), len(all_val_data.y_vals))
    gp_object = Type_1_GP_Emulator(all_gp_data, all_val_data, None, None, None, Kernel_enum(1), None, noise_std, None, 
                                retrain_GP, seed, normalize, None, None, None, None)
    
train_data, test_data = gp_object.set_train_test_data(1.0, seed)

0.024984087640608554


In [15]:
#Set up scalers
if gp_object.normalize:
    gp_object.scalerY.fit(gp_object.train_data.y_vals.reshape(-1,1))
    sclr = float(gp_object.scalerY.scale_)
    gp_object.scalerX = gp_object.scalerX.fit(gp_object.train_data_init)
else:
    sclr = 1.0

In [16]:
#Set up for sklearn kernel
if normalize:
    points = gp_object.scalerX.transform(gp_object.train_data_init)
else:
    points = gp_object.train_data_init

# Compute pairwise Euclidean distances between all points
pairwise_distances = np.sqrt(np.sum((points[:, None] - points) ** 2, axis=-1))

# Set the diagonal elements (self-distances) to infinity
np.fill_diagonal(pairwise_distances, np.inf)
# Mask out infinite values so that np.max will ignore them
pairwise_distances = np.ma.masked_invalid(pairwise_distances)

# Find the maximum and minimum distances between points
max_distance = np.max(pairwise_distances)
min_distance = np.min(pairwise_distances)

#Set max lenscl as the smaller of the max distance or 100
lenscl_1 = min(max_distance, 100)
#Set min lenscl as the larger of the min distance or 0.01
lenscl_2 = max(min_distance, 1e-2)
#Ensure min lenscl < max lenscl and that bounds are floats
min_lenscl = float(np.minimum(lenscl_1, lenscl_2))
max_lenscl = float(np.maximum(lenscl_1, lenscl_2))
lenscl_bnds = min_lenscl, max_lenscl

#Set the noise guess or allow gp to tune the noise parameter
if gp_object.noise_std is not None:
    #If we know the noise, use it
    noise_guess_org = (gp_object.noise_std/sclr)**2
    #Set the noise as the higher of noise_guess and 1e-10
    noise_guess = np.maximum(1e-10, noise_guess_org)
    noise_kern = WhiteKernel(noise_level=noise_guess, noise_level_bounds= "fixed")
else:
    #Otherwise, set the guess as 5% the taining data mean
    data_mean = np.abs(np.mean(gp_object.gp_sim_data.y_vals))
    noise_guess = data_mean*0.05/sclr
    #Set the min noise as 1% of the data mean or 1e-3 (minimum) and max as 10% or (1e-2 maximum)
    noise_min = max(data_mean*0.01/sclr, 1e-3)
    noise_max = noise_min*10 
    #Ensure the guess is in the bounds, if not, use the mean of the max and min
    if not noise_min <= noise_guess <= noise_max:
        noise_guess = (noise_min+noise_max)/2
    noise_kern = WhiteKernel(noise_level= noise_guess**2, noise_level_bounds= (noise_min**2, noise_max**2))

#If normalizing data
if gp_object.normalize:
    #Scale will be the average 
    train_y_scl = gp_object.scalerY.transform(gp_object.train_data.y_vals.reshape(-1,1))
    #Scaled bounds on C
    mse = sum(train_y_scl.flatten()**2)/len(train_y_scl.flatten())
    c_max = np.maximum(3, mse)
    c_min = np.minimum(1e-2, mse)
    c_bnds = (c_min,c_max)
    if c_bnds[0] < 1.0 < c_bnds[1]:
        c_guess = 1.0
    else:
        c_guess = (c_max + c_min) /2
else:
    #For unscaled data, this distance on the mean is dependent of the data
    c_bnds = (1e-2,1e2)
    c_guess = 1.0

cont_kern = ConstantKernel(constant_value = c_guess, constant_value_bounds=c_bnds)

if gp_object.kernel.value == 3: #RBF
    kernel = cont_kern*RBF(length_scale_bounds=(lenscl_bnds)) + noise_kern
elif gp_object.kernel.value == 2: #Matern 3/2
    kernel = cont_kern*Matern(length_scale_bounds=(lenscl_bnds), nu=1.5) + noise_kern 
else: #Matern 5/2
    kernel = cont_kern*Matern(length_scale_bounds=(lenscl_bnds), nu=2.5) + noise_kern 

In [17]:
#Make Sklearn and GPFLow Models
#GPFLOW
gpflow_model = gp_object.set_gp_model()
sklearn_kern = kernel

if isinstance(gp_object.lenscl, np.ndarray) and all(var is not None for var in gp_object.lenscl) and gp_object.outputscl != None:
    optimizer = None
elif isinstance(gp_object.lenscl, (float, int)) and gp_object.lenscl != None and gp_object.outputscl != None:
    optimizer = None
else:
    optimizer = "fmin_l_bfgs_b"
sklearn_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, n_restarts_optimizer=gp_object.retrain_GP, 
                                            random_state = gp_object.seed, optimizer = optimizer, normalize_y = not gp_object.normalize)

In [30]:
%%time
choice = "gpflow"
#Train gpflow model
if choice == "gpflow":
    gp_object.train_gp(gpflow_model)
else:
    #Train sklearn Model
    #Train GP
    #Preprocess Training data
    if gp_object.normalize == True:
        #Update scaler to be the fitted scaler. This scaler will change as the training data is updated
        gp_object.scalerX = gp_object.scalerX.fit(gp_object.feature_train_data)
        gp_object.scalerY = gp_object.scalerY.fit(gp_object.train_data.y_vals.reshape(-1,1))
        #Scale training data if necessary
        feature_train_data_scaled = gp_object.scalerX.transform(gp_object.feature_train_data)
        y_train_data_scaled = gp_object.scalerY.transform(gp_object.train_data.y_vals.reshape(-1,1))
    else:
        feature_train_data_scaled = gp_object.feature_train_data
        y_train_data_scaled = gp_object.train_data.y_vals.reshape(-1,1)

    #Fit GP Model
    fit_gp_model = sklearn_model.fit(feature_train_data_scaled, y_train_data_scaled)

    #Pull out kernel parameters after GP training
    opt_kern_params = fit_gp_model.kernel_
    outputscl_final = opt_kern_params.k1.k1.constant_value
    lenscl_final = opt_kern_params.k1.k2.length_scale
    noise_final = opt_kern_params.k2.noise_level

    #Put hyperparameters in a list
    trained_hyperparams = [lenscl_final, noise_final, outputscl_final] 

    #Assign self parameters
    gp_object.trained_hyperparams = trained_hyperparams
    gp_object.fit_gp_model = fit_gp_model

CPU times: user 35.1 s, sys: 3.14 s, total: 38.2 s
Wall time: 6.45 s


In [31]:
%%time
#Predict values
if choice == "gpflow":
    misc_gp_mean, misc_var_return = gp_object.eval_gp_mean_var_val(covar = False)
else:
    data = gp_object.featurize_data(all_val_data)
    if len(data.shape) < 2:
        data.reshape(1,-1)
    #scale eval_point if necessary
    if gp_object.normalize == True:
        eval_points = gp_object.scalerX.transform(data)
    else:
        eval_points = data
    
    #Evaluate GP given parameter set theta and state point value
    # print(self.fit_gp_model.kernel_)
    gp_mean_scl, gp_covar_scl = gp_object.fit_gp_model.predict(eval_points, return_cov=True)

    #Unscale gp_mean and gp_covariance
    if gp_object.normalize == True:
        gp_mean = gp_object.scalerY.inverse_transform(gp_mean_scl.reshape(-1,1)).flatten()
        gp_covar = float(gp_object.scalerY.scale_**2) * gp_covar_scl  
    else:
        gp_mean = gp_mean_scl
        gp_covar = gp_covar_scl
    
    gp_var = np.diag(gp_covar)
    misc_gp_mean = gp_mean
    misc_var_return = gp_var


CPU times: user 550 ms, sys: 131 ms, total: 681 ms
Wall time: 201 ms
